In [1]:
import pandas as pd

In [2]:
import json
filePath = r"C:\\Users\\Pat\\Desktop\\Projects\\LoLAramPredictor\\Game Info\\"

In [3]:
with open(filePath+'champion.json', encoding='utf8') as f:
    championData = json.load(f)

champions = pd.DataFrame(columns = ['championId', 'name', 'attack', 'defense', 'magic', 'difficulty', 'primaryClass', 'secondaryClass', 'partype', 'hp', 'hpperlevel', 'mp', 'mpperlevel', 'movespeed', 'armor', 'armorperlevel', 'spellblock', 'spellblockperlevel', 'attackrange', 'hpregen', 'hpregenperlevel', 'mpregen', 'mpregenperlevel', 'crit', 'critperlevel', 'attackdamage', 'attackdamageperlevel', 'attackspeedperlevel', 'attackspeed'])

for k, v in championData['data'].items():
    champions = champions.append({'championId': int(v['key']),
                        'name': v['name'], 
                        'attack': v['info']['attack'], 
                        'defense': v['info']['defense'], 
                        'magic': v['info']['magic'], 
                        'difficulty': v['info']['difficulty'], 
                        'primaryClass': v['tags'][0], 
                        'secondaryClass': v['tags'][1] if len(v['tags']) > 1 else 'None', 
                        'partype': v['partype'], 
                        'hp': v['stats']['hp'], 
                        'hpperlevel': v['stats']['hpperlevel'], 
                        'mp': v['stats']['mp'], 
                        'mpperlevel': v['stats']['mpperlevel'], 
                        'movespeed': v['stats']['movespeed'], 
                        'armor': v['stats']['armor'], 
                        'armorperlevel': v['stats']['armorperlevel'], 
                        'spellblock': v['stats']['spellblock'], 
                        'spellblockperlevel': v['stats']['spellblockperlevel'], 
                        'attackrange': v['stats']['attackrange'], 
                        'hpregen': v['stats']['hpregen'], 
                        'hpregenperlevel': v['stats']['hpregenperlevel'], 
                        'mpregen': v['stats']['mpregen'], 
                        'mpregenperlevel': v['stats']['mpregenperlevel'], 
                        'crit': v['stats']['crit'], 
                        'critperlevel': v['stats']['critperlevel'], 
                        'attackdamage': v['stats']['attackdamage'], 
                        'attackdamageperlevel': v['stats']['attackdamageperlevel'], 
                        'attackspeedperlevel': v['stats']['attackspeedperlevel'], 
                        'attackspeed': v['stats']['attackspeed']}, ignore_index=True)

champions.set_index('championId', inplace=True)
champions

,name,attack,defense,magic,difficulty,primaryClass,secondaryClass,partype,hp,hpperlevel,...,hpregen,hpregenperlevel,mpregen,mpregenperlevel,crit,critperlevel,attackdamage,attackdamageperlevel,attackspeedperlevel,attackspeed
championId,,,,,,,,,,,,,,,,,,,,,
266,Aatrox,8,4,3,4,Fighter,Tank,Blood Well,580,90,...,3,1,0,0,0,0,60,5,2.500,0.651
103,Ahri,3,4,8,5,Mage,Assassin,Mana,526,92,...,5.5,0.6,8,0.8,0,0,53.04,3,2.000,0.668
84,Akali,5,3,8,7,Assassin,None,Energy,575,95,...,8,0.5,50,0,0,0,62.4,3.3,3.200,0.625
12,Alistar,6,9,5,7,Tank,Support,Mana,600,106,...,8.5,0.85,8.5,0.8,0,0,62,3.75,2.125,0.625
32,Amumu,2,6,8,3,Tank,Mage,Mana,613.12,84,...,9,0.85,7.382,0.525,0,0,53.38,3.8,2.180,0.736
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
238,Zed,9,2,1,7,Assassin,None,Energy,584,85,...,7,0.65,50,0,0,0,63,3.4,3.300,0.651
115,Ziggs,2,4,9,4,Mage,None,Mana,536,92,...,6.5,0.6,8,0.8,0,0,54.208,3.1,2.000,0.656
26,Zilean,2,5,8,6,Support,Mage,Mana,504,82,...,5.5,0.5,11.335,0.8,0,0,51.64,3,2.130,0.625


In [ ]:
import asyncpg
import nest_asyncio
import asyncio
from config import dbConfig

nest_asyncio.apply()
loop = asyncio.get_event_loop()

In [ ]:
async def fetch_as_dataframe(con: asyncpg.Connection, query: str, *args):
    stmt = await con.prepare(query)
    columns = [a.name for a in stmt.get_attributes()]
    data = await stmt.fetch()
    return pd.DataFrame(data, columns=columns)

async def run():
    connection = await asyncpg.connect(user=dbConfig['user'],
                                password=dbConfig['password'],
                                host=dbConfig['host'],
                                port=dbConfig['port'],
                                database=dbConfig['database'])
    async with connection.transaction():
        # allGamesQuery = f'SELECT "gameVersion", "gameMode", COUNT(1) \
        #                 FROM "Matches" \
        #                 GROUP BY "gameVersion", "gameMode"\
        #                 ORDER BY "gameVersion" ASC;'
        patchCountQuery = f'SELECT (string_to_array("gameVersion", \'.\'))[1] || \'.\' || (string_to_array("gameVersion", \'.\'))[2] AS patch, "gameMode" , COUNT(1) \
                        FROM "Matches" \
                        GROUP BY patch, "gameMode" \
                        ORDER BY "gameMode", patch;'
        patchCount_df = await fetch_as_dataframe(connection, patchCountQuery)

    return patchCount_df

patchCount_df = loop.run_until_complete(run())
patchCount_df.head(50)

In [ ]:
async def fetch_as_dataframe(con: asyncpg.Connection, query: str, *args):
    stmt = await con.prepare(query)
    columns = [a.name for a in stmt.get_attributes()]
    data = await stmt.fetch()
    return pd.DataFrame(data, columns=columns)

async def run():
    connection = await asyncpg.connect(user=dbConfig['user'],
                                password=dbConfig['password'],
                                host=dbConfig['host'],
                                port=dbConfig['port'],
                                database=dbConfig['database'])
    async with connection.transaction():
        rankQuery = f'SELECT "highestAchievedSeasonTier", COUNT(1) \
                        FROM "Participant_Stats"\
                        GROUP BY "highestAchievedSeasonTier" \
                        ORDER BY "highestAchievedSeasonTier" ASC;'
        rank_df = await fetch_as_dataframe(connection, rankQuery)

    return rank_df

rank_df = loop.run_until_complete(run())
rank_df.head(10)

In [ ]:
# async def fetch_as_dataframe(con: asyncpg.Connection, query: str, *args):
#     stmt = await con.prepare(query)
#     columns = [a.name for a in stmt.get_attributes()]
#     data = await stmt.fetch()
#     return pd.DataFrame(data, columns=columns)

# async def run():
#     connection = await asyncpg.connect(user=dbConfig['user'],
#                                 password=dbConfig['password'],
#                                 host=dbConfig['host'],
#                                 port=dbConfig['port'],
#                                 database=dbConfig['database'])
#     async with connection.transaction():
#         allGamesQuery = f'SELECT p1."gameId"\
#         , MAX(CASE p2.Ranking WHEN 1 THEN p2."championId" END) AS champ_1 \
#         , MAX(CASE p2.Ranking WHEN 2 THEN p2."championId" END) AS champ_2 \
#         , MAX(CASE p2.Ranking WHEN 3 THEN p2."championId" END) AS champ_3 \
#         , MAX(CASE p2.Ranking WHEN 4 THEN p2."championId" END) AS champ_4 \
#         , MAX(CASE p2.Ranking WHEN 5 THEN p2."championId" END) AS champ_5 \
#         , MAX(CASE p2.Ranking WHEN 6 THEN p2."championId" END) AS champ_6 \
#         , MAX(CASE p2.Ranking WHEN 7 THEN p2."championId" END) AS champ_7 \
#         , MAX(CASE p2.Ranking WHEN 8 THEN p2."championId" END) AS champ_8 \
#         , MAX(CASE p2.Ranking WHEN 9 THEN p2."championId" END) AS champ_9 \
#         , MAX(CASE p2.Ranking WHEN 10 THEN p2."championId" END) AS champ_10 \
#         , MAX(CASE p2.Ranking WHEN 1 THEN p1."tier" END) AS "tier_1" \
#         , MAX(CASE p2.Ranking WHEN 2 THEN p1."tier" END) AS "tier_2" \
#         , MAX(CASE p2.Ranking WHEN 3 THEN p1."tier" END) AS "tier_3" \
#         , MAX(CASE p2.Ranking WHEN 4 THEN p1."tier" END) AS "tier_4" \
#         , MAX(CASE p2.Ranking WHEN 5 THEN p1."tier" END) AS "tier_5" \
#         , MAX(CASE p2.Ranking WHEN 6 THEN p1."tier" END) AS "tier_6" \
#         , MAX(CASE p2.Ranking WHEN 7 THEN p1."tier" END) AS "tier_7" \
#         , MAX(CASE p2.Ranking WHEN 8 THEN p1."tier" END) AS "tier_8" \
#         , MAX(CASE p2.Ranking WHEN 9 THEN p1."tier" END) AS "tier_9" \
#         , MAX(CASE p2.Ranking WHEN 10 THEN p1."tier" END) AS "tier_10" \
#         , MAX(CASE p2.Ranking WHEN 1 THEN p1."rank" END) AS "rank_1" \
#         , MAX(CASE p2.Ranking WHEN 2 THEN p1."rank" END) AS "rank_2" \
#         , MAX(CASE p2.Ranking WHEN 3 THEN p1."rank" END) AS "rank_3" \
#         , MAX(CASE p2.Ranking WHEN 4 THEN p1."rank" END) AS "rank_4" \
#         , MAX(CASE p2.Ranking WHEN 5 THEN p1."rank" END) AS "rank_5" \
#         , MAX(CASE p2.Ranking WHEN 6 THEN p1."rank" END) AS "rank_6" \
#         , MAX(CASE p2.Ranking WHEN 7 THEN p1."rank" END) AS "rank_7" \
#         , MAX(CASE p2.Ranking WHEN 8 THEN p1."rank" END) AS "rank_8" \
#         , MAX(CASE p2.Ranking WHEN 9 THEN p1."rank" END) AS "rank_9" \
#         , MAX(CASE p2.Ranking WHEN 10 THEN p1."rank" END) AS "rank_10" \
#         , MAX(CASE p2.Ranking WHEN 1 THEN p1."rankWins" END) AS "rankWins_1" \
#         , MAX(CASE p2.Ranking WHEN 2 THEN p1."rankWins" END) AS "rankWins_2" \
#         , MAX(CASE p2.Ranking WHEN 3 THEN p1."rankWins" END) AS "rankWins_3" \
#         , MAX(CASE p2.Ranking WHEN 4 THEN p1."rankWins" END) AS "rankWins_4" \
#         , MAX(CASE p2.Ranking WHEN 5 THEN p1."rankWins" END) AS "rankWins_5" \
#         , MAX(CASE p2.Ranking WHEN 6 THEN p1."rankWins" END) AS "rankWins_6" \
#         , MAX(CASE p2.Ranking WHEN 7 THEN p1."rankWins" END) AS "rankWins_7" \
#         , MAX(CASE p2.Ranking WHEN 8 THEN p1."rankWins" END) AS "rankWins_8" \
#         , MAX(CASE p2.Ranking WHEN 9 THEN p1."rankWins" END) AS "rankWins_9" \
#         , MAX(CASE p2.Ranking WHEN 10 THEN p1."rankWins" END) AS "rankWins_10" \
#         , MAX(CASE p2.Ranking WHEN 1 THEN p1."rankLosses" END) AS "rankLosses_1" \
#         , MAX(CASE p2.Ranking WHEN 2 THEN p1."rankLosses" END) AS "rankLosses_2" \
#         , MAX(CASE p2.Ranking WHEN 3 THEN p1."rankLosses" END) AS "rankLosses_3" \
#         , MAX(CASE p2.Ranking WHEN 4 THEN p1."rankLosses" END) AS "rankLosses_4" \
#         , MAX(CASE p2.Ranking WHEN 5 THEN p1."rankLosses" END) AS "rankLosses_5" \
#         , MAX(CASE p2.Ranking WHEN 6 THEN p1."rankLosses" END) AS "rankLosses_6" \
#         , MAX(CASE p2.Ranking WHEN 7 THEN p1."rankLosses" END) AS "rankLosses_7" \
#         , MAX(CASE p2.Ranking WHEN 8 THEN p1."rankLosses" END) AS "rankLosses_8" \
#         , MAX(CASE p2.Ranking WHEN 9 THEN p1."rankLosses" END) AS "rankLosses_9" \
#         , MAX(CASE p2.Ranking WHEN 10 THEN p1."rankLosses" END) AS "rankLosses_10" \
#         , MAX(CASE p2.Ranking WHEN 1 THEN p1."championPoints" END) AS "champPts_1" \
#         , MAX(CASE p2.Ranking WHEN 2 THEN p1."championPoints" END) AS "champPts_2" \
#         , MAX(CASE p2.Ranking WHEN 3 THEN p1."championPoints" END) AS "champPts_3" \
#         , MAX(CASE p2.Ranking WHEN 4 THEN p1."championPoints" END) AS "champPts_4" \
#         , MAX(CASE p2.Ranking WHEN 5 THEN p1."championPoints" END) AS "champPts_5" \
#         , MAX(CASE p2.Ranking WHEN 6 THEN p1."championPoints" END) AS "champPts_6" \
#         , MAX(CASE p2.Ranking WHEN 7 THEN p1."championPoints" END) AS "champPts_7" \
#         , MAX(CASE p2.Ranking WHEN 8 THEN p1."championPoints" END) AS "champPts_8" \
#         , MAX(CASE p2.Ranking WHEN 9 THEN p1."championPoints" END) AS "champPts_9" \
#         , MAX(CASE p2.Ranking WHEN 10 THEN p1."championPoints" END) AS "champPts_10" \
#         , MAX(CASE p2.Ranking WHEN 1 THEN p2."spell1Id" END) AS "spell1Id_1" \
#         , MAX(CASE p2.Ranking WHEN 2 THEN p2."spell1Id" END) AS "spell1Id_2" \
#         , MAX(CASE p2.Ranking WHEN 3 THEN p2."spell1Id" END) AS "spell1Id_3" \
#         , MAX(CASE p2.Ranking WHEN 4 THEN p2."spell1Id" END) AS "spell1Id_4" \
#         , MAX(CASE p2.Ranking WHEN 5 THEN p2."spell1Id" END) AS "spell1Id_5" \
#         , MAX(CASE p2.Ranking WHEN 6 THEN p2."spell1Id" END) AS "spell1Id_6" \
#         , MAX(CASE p2.Ranking WHEN 7 THEN p2."spell1Id" END) AS "spell1Id_7" \
#         , MAX(CASE p2.Ranking WHEN 8 THEN p2."spell1Id" END) AS "spell1Id_8" \
#         , MAX(CASE p2.Ranking WHEN 9 THEN p2."spell1Id" END) AS "spell1Id_9" \
#         , MAX(CASE p2.Ranking WHEN 10 THEN p2."spell1Id" END) AS "spell1Id_10" \
#         , MAX(CASE p2.Ranking WHEN 1 THEN p2."spell2Id" END) AS "spell2Id_1" \
#         , MAX(CASE p2.Ranking WHEN 2 THEN p2."spell2Id" END) AS "spell2Id_2" \
#         , MAX(CASE p2.Ranking WHEN 3 THEN p2."spell2Id" END) AS "spell2Id_3" \
#         , MAX(CASE p2.Ranking WHEN 4 THEN p2."spell2Id" END) AS "spell2Id_4" \
#         , MAX(CASE p2.Ranking WHEN 5 THEN p2."spell2Id" END) AS "spell2Id_5" \
#         , MAX(CASE p2.Ranking WHEN 6 THEN p2."spell2Id" END) AS "spell2Id_6" \
#         , MAX(CASE p2.Ranking WHEN 7 THEN p2."spell2Id" END) AS "spell2Id_7" \
#         , MAX(CASE p2.Ranking WHEN 8 THEN p2."spell2Id" END) AS "spell2Id_8" \
#         , MAX(CASE p2.Ranking WHEN 9 THEN p2."spell2Id" END) AS "spell2Id_9" \
#         , MAX(CASE p2.Ranking WHEN 10 THEN p2."spell2Id" END) AS "spell2Id_10" \
#         , MAX(CASE p2.Ranking WHEN 1 THEN p2."perk0" END) AS perk0_1 \
#         , MAX(CASE p2.Ranking WHEN 2 THEN p2."perk0" END) AS perk0_2 \
#         , MAX(CASE p2.Ranking WHEN 3 THEN p2."perk0" END) AS perk0_3 \
#         , MAX(CASE p2.Ranking WHEN 4 THEN p2."perk0" END) AS perk0_4 \
#         , MAX(CASE p2.Ranking WHEN 5 THEN p2."perk0" END) AS perk0_5 \
#         , MAX(CASE p2.Ranking WHEN 6 THEN p2."perk0" END) AS perk0_6 \
#         , MAX(CASE p2.Ranking WHEN 7 THEN p2."perk0" END) AS perk0_7 \
#         , MAX(CASE p2.Ranking WHEN 8 THEN p2."perk0" END) AS perk0_8 \
#         , MAX(CASE p2.Ranking WHEN 9 THEN p2."perk0" END) AS perk0_9 \
#         , MAX(CASE p2.Ranking WHEN 10 THEN p2."perk0" END) AS perk0_10 \
#         , MAX(CASE p2.Ranking WHEN 1 THEN p2."perk1" END) AS perk1_1 \
#         , MAX(CASE p2.Ranking WHEN 2 THEN p2."perk1" END) AS perk1_2 \
#         , MAX(CASE p2.Ranking WHEN 3 THEN p2."perk1" END) AS perk1_3 \
#         , MAX(CASE p2.Ranking WHEN 4 THEN p2."perk1" END) AS perk1_4 \
#         , MAX(CASE p2.Ranking WHEN 5 THEN p2."perk1" END) AS perk1_5 \
#         , MAX(CASE p2.Ranking WHEN 6 THEN p2."perk1" END) AS perk1_6 \
#         , MAX(CASE p2.Ranking WHEN 7 THEN p2."perk1" END) AS perk1_7 \
#         , MAX(CASE p2.Ranking WHEN 8 THEN p2."perk1" END) AS perk1_8 \
#         , MAX(CASE p2.Ranking WHEN 9 THEN p2."perk1" END) AS perk1_9 \
#         , MAX(CASE p2.Ranking WHEN 10 THEN p2."perk1" END) AS perk1_10 \
#         , MAX(CASE p2.Ranking WHEN 1 THEN p2."perk2" END) AS perk2_1 \
#         , MAX(CASE p2.Ranking WHEN 2 THEN p2."perk2" END) AS perk2_2 \
#         , MAX(CASE p2.Ranking WHEN 3 THEN p2."perk2" END) AS perk2_3 \
#         , MAX(CASE p2.Ranking WHEN 4 THEN p2."perk2" END) AS perk2_4 \
#         , MAX(CASE p2.Ranking WHEN 5 THEN p2."perk2" END) AS perk2_5 \
#         , MAX(CASE p2.Ranking WHEN 6 THEN p2."perk2" END) AS perk2_6 \
#         , MAX(CASE p2.Ranking WHEN 7 THEN p2."perk2" END) AS perk2_7 \
#         , MAX(CASE p2.Ranking WHEN 8 THEN p2."perk2" END) AS perk2_8 \
#         , MAX(CASE p2.Ranking WHEN 9 THEN p2."perk2" END) AS perk2_9 \
#         , MAX(CASE p2.Ranking WHEN 10 THEN p2."perk2" END) AS perk2_10 \
#         , MAX(CASE p2.Ranking WHEN 1 THEN p2."perk3" END) AS perk3_1 \
#         , MAX(CASE p2.Ranking WHEN 2 THEN p2."perk3" END) AS perk3_2 \
#         , MAX(CASE p2.Ranking WHEN 3 THEN p2."perk3" END) AS perk3_3 \
#         , MAX(CASE p2.Ranking WHEN 4 THEN p2."perk3" END) AS perk3_4 \
#         , MAX(CASE p2.Ranking WHEN 5 THEN p2."perk3" END) AS perk3_5 \
#         , MAX(CASE p2.Ranking WHEN 6 THEN p2."perk3" END) AS perk3_6 \
#         , MAX(CASE p2.Ranking WHEN 7 THEN p2."perk3" END) AS perk3_7 \
#         , MAX(CASE p2.Ranking WHEN 8 THEN p2."perk3" END) AS perk3_8 \
#         , MAX(CASE p2.Ranking WHEN 9 THEN p2."perk3" END) AS perk3_9 \
#         , MAX(CASE p2.Ranking WHEN 10 THEN p2."perk3" END) AS perk3_10 \
#         , MAX(CASE p2.Ranking WHEN 1 THEN p2."perk4" END) AS perk4_1 \
#         , MAX(CASE p2.Ranking WHEN 2 THEN p2."perk4" END) AS perk4_2 \
#         , MAX(CASE p2.Ranking WHEN 3 THEN p2."perk4" END) AS perk4_3 \
#         , MAX(CASE p2.Ranking WHEN 4 THEN p2."perk4" END) AS perk4_4 \
#         , MAX(CASE p2.Ranking WHEN 5 THEN p2."perk4" END) AS perk4_5 \
#         , MAX(CASE p2.Ranking WHEN 6 THEN p2."perk4" END) AS perk4_6 \
#         , MAX(CASE p2.Ranking WHEN 7 THEN p2."perk4" END) AS perk4_7 \
#         , MAX(CASE p2.Ranking WHEN 8 THEN p2."perk4" END) AS perk4_8 \
#         , MAX(CASE p2.Ranking WHEN 9 THEN p2."perk4" END) AS perk4_9 \
#         , MAX(CASE p2.Ranking WHEN 10 THEN p2."perk4" END) AS perk4_10 \
#         , MAX(CASE p2.Ranking WHEN 1 THEN p2."perk5" END) AS perk5_1 \
#         , MAX(CASE p2.Ranking WHEN 2 THEN p2."perk5" END) AS perk5_2 \
#         , MAX(CASE p2.Ranking WHEN 3 THEN p2."perk5" END) AS perk5_3 \
#         , MAX(CASE p2.Ranking WHEN 4 THEN p2."perk5" END) AS perk5_4 \
#         , MAX(CASE p2.Ranking WHEN 5 THEN p2."perk5" END) AS perk5_5 \
#         , MAX(CASE p2.Ranking WHEN 6 THEN p2."perk5" END) AS perk5_6 \
#         , MAX(CASE p2.Ranking WHEN 7 THEN p2."perk5" END) AS perk5_7 \
#         , MAX(CASE p2.Ranking WHEN 8 THEN p2."perk5" END) AS perk5_8 \
#         , MAX(CASE p2.Ranking WHEN 9 THEN p2."perk5" END) AS perk5_9 \
#         , MAX(CASE p2.Ranking WHEN 10 THEN p2."perk5" END) AS perk5_10 \
#         , MAX(p1."gameMode") AS "gameMode" \
#         , MAX(p1."gameVersion") AS "gameVersion" \
#         , BOOL_OR(CASE p2.Ranking WHEN 1 THEN p2."win" END) AS "blueTeamWin" \
#     FROM (SELECT p."gameId", p."teamColor", p."championId", c."championPoints", r."tier", r."rank", r."wins" AS "rankWins", r."losses" AS "rankLosses", p."spell1Id", p."spell2Id", p."perk0", p."perk1", p."perk2", p."perk3", p."perk4", p."perk5", m."gameMode", m."gameVersion", p."win" \
#                         FROM "Participant_Stats" AS p \
#                         LEFT JOIN "Matches" AS m \
#                         ON p."gameId" = m."gameId" \
#                         LEFT JOIN "Champion_Mastery" AS c \
#                         ON p."summonerName" = c."summonerName" AND p."championId" = c."championId" \
#                         LEFT JOIN (SELECT "summonerName", "queueType", "tier", "rank", "wins", losses \
#                             FROM "Rank" \
#                             WHERE "queueType" = \'RANKED_SOLO_5x5\') AS r \
#                         ON p."summonerName" = r."summonerName") AS p1 \
#     LEFT JOIN ( \
#         SELECT *, ROW_NUMBER() OVER (PARTITION by "gameId" ORDER BY "teamColor", "championId") AS Ranking \
#         FROM "Participant_Stats" \
#     ) AS p2 ON p2."gameId"= p1."gameId" AND p2."championId" = p1."championId"\
#     WHERE "gameMode" = \'ARAM\' \
#     GROUP BY p1."gameId";'
#         groupGames_df = await fetch_as_dataframe(connection, allGamesQuery)

#     return groupGames_df

# groupGames_df = loop.run_until_complete(run())
# groupGames_df.head(10)

In [ ]:
# print(groupGames_df[['gameId', 'champ_1', 'tier_1', 'rank_1', 'rankWins_1', 'rankLosses_1', 'champPts_1', 'spell1Id_1', 'spell2Id_1', 'perk0_1', 'perk1_1', 'perk2_1', 'perk3_1', 'perk4_1', 'perk5_1', 'gameMode', 'gameVersion', 'blueTeamWin']].head(50))

In [ ]:
# print(groupGames_df[['gameId', 'champ_1', 'tier_1', 'rank_1', 'rankWins_1', 'rankLosses_1', 'champPts_1', 'spell1Id_1', 'spell2Id_1', 'perk0_1', 'perk1_1', 'perk2_1', 'perk3_1', 'perk4_1', 'perk5_1', 'gameMode', 'gameVersion', 'blueTeamWin']].tail(50))

In [ ]:
async def run():
    connection = await asyncpg.connect(user=dbConfig['user'],
                                password=dbConfig['password'],
                                host=dbConfig['host'],
                                port=dbConfig['port'],
                                database=dbConfig['database'])
    async with connection.transaction():
        refreshQuery = f'REFRESH MATERIALIZED VIEW "games_mat_view";'
        status = await connection.execute(refreshQuery)
    return status

status = loop.run_until_complete(run())
print(status)

In [ ]:
async def fetch_as_dataframe(con: asyncpg.Connection, query: str, *args):
    stmt = await con.prepare(query)
    columns = [a.name for a in stmt.get_attributes()]
    data = await stmt.fetch()
    return pd.DataFrame(data, columns=columns)

async def run():
    connection = await asyncpg.connect(user=dbConfig['user'],
                                password=dbConfig['password'],
                                host=dbConfig['host'],
                                port=dbConfig['port'],
                                database=dbConfig['database'])
    async with connection.transaction():
        matViewQuery = f'SELECT * FROM "games_mat_view" ORDER BY "gameId" DESC'
        df = await fetch_as_dataframe(connection, matViewQuery)

    return df.drop(columns=['gameMode'])

df = loop.run_until_complete(run())
df.head(10)

In [ ]:
print(df.head(10))
print(df.columns.to_list())

In [ ]:
df.info(verbose=True, null_counts=True)

In [ ]:
df1 = df
df1 = df1.dropna(axis=0, subset=['gameId', 'champ_1', 'champ_2', 'champ_3', 'champ_4', 'champ_5', 'champ_6', 'champ_7', 'champ_8', 'champ_9', 'champ_10', 'spell1Id_1', 'spell1Id_2', 'spell1Id_3', 'spell1Id_4', 'spell1Id_5', 'spell1Id_6', 'spell1Id_7', 'spell1Id_8', 'spell1Id_9', 'spell1Id_10', 'spell2Id_1', 'spell2Id_2', 'spell2Id_3', 'spell2Id_4', 'spell2Id_5', 'spell2Id_6', 'spell2Id_7', 'spell2Id_8', 'spell2Id_9', 'spell2Id_10', 'perk0_1', 'perk0_2', 'perk0_3', 'perk0_4', 'perk0_5', 'perk0_6', 'perk0_7', 'perk0_8', 'perk0_9', 'perk0_10', 'perk1_1', 'perk1_2', 'perk1_3', 'perk1_4', 'perk1_5', 'perk1_6', 'perk1_7', 'perk1_8', 'perk1_9', 'perk1_10', 'perk2_1', 'perk2_2', 'perk2_3', 'perk2_4', 'perk2_5', 'perk2_6', 'perk2_7', 'perk2_8', 'perk2_9', 'perk2_10', 'perk3_1', 'perk3_2', 'perk3_3', 'perk3_4', 'perk3_5', 'perk3_6', 'perk3_7', 'perk3_8', 'perk3_9', 'perk3_10', 'perk4_1', 'perk4_2', 'perk4_3', 'perk4_4', 'perk4_5', 'perk4_6', 'perk4_7', 'perk4_8', 'perk4_9', 'perk4_10', 'perk5_1', 'perk5_2', 'perk5_3', 'perk5_4', 'perk5_5', 'perk5_6', 'perk5_7', 'perk5_8', 'perk5_9', 'perk5_10', 'gameVersion', 'blueTeamWin'], how='any')

In [ ]:
df1 = df1.astype({'gameId': 'uint64', 'champ_1': 'uint16', 'champ_2': 'uint16', 'champ_3': 'uint16', 'champ_4': 'uint16', 'champ_5': 'uint16', 'champ_6': 'uint16', 'champ_7': 'uint16', 'champ_8': 'uint16', 'champ_9': 'uint16', 'champ_10': 'uint16', 'tier_1': 'str', 'tier_2': 'str', 'tier_3': 'str', 'tier_4': 'str', 'tier_5': 'str', 'tier_6': 'str', 'tier_7': 'str', 'tier_8': 'str', 'tier_9': 'str', 'tier_10': 'str', 'rank_1': 'str', 'rank_2': 'str', 'rank_3': 'str', 'rank_4': 'str', 'rank_5': 'str', 'rank_6': 'str', 'rank_7': 'str', 'rank_8': 'str', 'rank_9': 'str', 'rank_10': 'str', 'rankWins_1': 'uint32', 'rankWins_2': 'uint32', 'rankWins_3': 'uint32', 'rankWins_4': 'uint32', 'rankWins_5': 'uint32', 'rankWins_6': 'uint32', 'rankWins_7': 'uint32', 'rankWins_8': 'uint32', 'rankWins_9': 'uint32', 'rankWins_10': 'uint32', 'rankLosses_1': 'uint32', 'rankLosses_2': 'uint32', 'rankLosses_3': 'uint32', 'rankLosses_4': 'uint32', 'rankLosses_5': 'uint32', 'rankLosses_6': 'uint32', 'rankLosses_7': 'uint32', 'rankLosses_8': 'uint32', 'rankLosses_9': 'uint32', 'rankLosses_10': 'uint32', 'champPts_1': 'uint64', 'champPts_2': 'uint64', 'champPts_3': 'uint64', 'champPts_4': 'uint64', 'champPts_5': 'uint64', 'champPts_6': 'uint64', 'champPts_7': 'uint64', 'champPts_8': 'uint64', 'champPts_9': 'uint64', 'champPts_10': 'uint64', 'spell1Id_1': 'uint8', 'spell1Id_2': 'uint8', 'spell1Id_3': 'uint8', 'spell1Id_4': 'uint8', 'spell1Id_5': 'uint8', 'spell1Id_6': 'uint8', 'spell1Id_7': 'uint8', 'spell1Id_8': 'uint8', 'spell1Id_9': 'uint8', 'spell1Id_10': 'uint8', 'spell2Id_1': 'uint8', 'spell2Id_2': 'uint8', 'spell2Id_3': 'uint8', 'spell2Id_4': 'uint8', 'spell2Id_5': 'uint8', 'spell2Id_6': 'uint8', 'spell2Id_7': 'uint8', 'spell2Id_8': 'uint8', 'spell2Id_9': 'uint8', 'spell2Id_10': 'uint8', 'perk0_1': 'uint16', 'perk0_2': 'uint16', 'perk0_3': 'uint16', 'perk0_4': 'uint16', 'perk0_5': 'uint16', 'perk0_6': 'uint16', 'perk0_7': 'uint16', 'perk0_8': 'uint16', 'perk0_9': 'uint16', 'perk0_10': 'uint16', 'perk1_1': 'uint16', 'perk1_2': 'uint16', 'perk1_3': 'uint16', 'perk1_4': 'uint16', 'perk1_5': 'uint16', 'perk1_6': 'uint16', 'perk1_7': 'uint16', 'perk1_8': 'uint16', 'perk1_9': 'uint16', 'perk1_10': 'uint16', 'perk2_1': 'uint16', 'perk2_2': 'uint16', 'perk2_3': 'uint16', 'perk2_4': 'uint16', 'perk2_5': 'uint16', 'perk2_6': 'uint16', 'perk2_7': 'uint16', 'perk2_8': 'uint16', 'perk2_9': 'uint16', 'perk2_10': 'uint16', 'perk3_1': 'uint16', 'perk3_2': 'uint16', 'perk3_3': 'uint16', 'perk3_4': 'uint16', 'perk3_5': 'uint16', 'perk3_6': 'uint16', 'perk3_7': 'uint16', 'perk3_8': 'uint16', 'perk3_9': 'uint16', 'perk3_10': 'uint16', 'perk4_1': 'uint16', 'perk4_2': 'uint16', 'perk4_3': 'uint16', 'perk4_4': 'uint16', 'perk4_5': 'uint16', 'perk4_6': 'uint16', 'perk4_7': 'uint16', 'perk4_8': 'uint16', 'perk4_9': 'uint16', 'perk4_10': 'uint16', 'perk5_1': 'uint16', 'perk5_2': 'uint16', 'perk5_3': 'uint16', 'perk5_4': 'uint16', 'perk5_5': 'uint16', 'perk5_6': 'uint16', 'perk5_7': 'uint16', 'perk5_8': 'uint16', 'perk5_9': 'uint16', 'perk5_10': 'uint16', 'gameVersion': 'category', 'blueTeamWin': 'bool'})

In [ ]:
df1.info(verbose=True, null_counts=True)

In [ ]:
# df1.to_pickle('game_data_clean.pkl')

In [4]:
import pandas as pd
# df1 = pd.read_pickle('game_data.pkl')
df1 = pd.read_pickle('game_data_clean.pkl')

In [5]:
df1.fillna(value={'tier_1': 'UNRANKED', 'tier_2': 'UNRANKED', 'tier_3': 'UNRANKED', 'tier_4': 'UNRANKED', 'tier_5': 'UNRANKED', 'tier_6': 'UNRANKED', 'tier_7': 'UNRANKED', 'tier_8': 'UNRANKED', 'tier_9': 'UNRANKED', 'tier_10': 'UNRANKED', 'rank_1': 'UNRANKED', 'rank_2': 'UNRANKED', 'rank_3': 'UNRANKED', 'rank_4': 'UNRANKED', 'rank_5': 'UNRANKED', 'rank_6': 'UNRANKED', 'rank_7': 'UNRANKED', 'rank_8': 'UNRANKED', 'rank_9': 'UNRANKED', 'rank_10': 'UNRANKED', 'rankWins_1': 0, 'rankWins_2': 0, 'rankWins_3': 0, 'rankWins_4': 0, 'rankWins_5': 0, 'rankWins_6': 0, 'rankWins_7': 0, 'rankWins_8': 0, 'rankWins_9': 0, 'rankWins_10': 0, 'rankLosses_1': 0, 'rankLosses_2': 0, 'rankLosses_3': 0, 'rankLosses_4': 0, 'rankLosses_5': 0, 'rankLosses_6': 0, 'rankLosses_7': 0, 'rankLosses_8': 0, 'rankLosses_9': 0, 'rankLosses_10': 0, 'champPts_1': 0, 'champPts_2': 0, 'champPts_3': 0, 'champPts_4': 0, 'champPts_5': 0, 'champPts_6': 0, 'champPts_7': 0, 'champPts_8': 0, 'champPts_9': 0, 'champPts_10': 0}, inplace=True)

In [6]:
for r in ['rank_1', 'rank_2', 'rank_3', 'rank_4', 'rank_5', 'rank_6', 'rank_7', 'rank_8', 'rank_9', 'rank_10']:
    if r == 'rank_10':
        df1.loc[df1[r] != 'UNRANKED', r] = df1.loc[df1[r] != 'UNRANKED', f'tier_10'].astype(str) + ' ' + df1.loc[df1[r] != 'UNRANKED', r].astype(str)
    else:
        df1.loc[df1[r] != 'UNRANKED', r] = df1.loc[df1[r] != 'UNRANKED', f'tier_{r[-1]}'].astype(str) + ' ' + df1.loc[df1[r] != 'UNRANKED', r].astype(str)
    df1[r] = pd.Categorical(df1[r], ['UNRANKED', 'IRON IV', 'IRON III', 'IRON II', 'IRON I', 'BRONZE IV', 'BRONZE III', 'BRONZE II', 'BRONZE I', 'SILVER IV', 'SILVER III', 'SILVER II', 'SILVER I', 'GOLD IV', 'GOLD III', 'GOLD II', 'GOLD I', 'PLATINUM IV', 'PLATINUM III', 'PLATINUM II', 'PLATINUM I', 'DIAMOND IV', 'DIAMOND III', 'DIAMOND II', 'DIAMOND I', 'MASTER I', 'GRANDMASTER I', 'CHALLENGER I'], ordered=True) 

In [7]:
for c in ['champ_1', 'champ_2', 'champ_3', 'champ_4', 'champ_5', 'champ_6', 'champ_7', 'champ_8', 'champ_9', 'champ_10']:
    if c == 'champ_10':
        suffix = f'_10'
    else:
        suffix = f'_{c[-1]}'
    df1 = pd.merge(df1, champions[['primaryClass', 'secondaryClass']].add_suffix(suffix), how='left', left_on=[c], right_on=['championId'], left_index=False, right_index=True)

In [8]:
df1 = df1.drop(['tier_1', 'tier_2', 'tier_3', 'tier_4', 'tier_5', 'tier_6', 'tier_7', 'tier_8', 'tier_9', 'tier_10'], axis=1)

In [9]:
df1 = df1.astype({'gameId': 'uint64', 'champ_1': 'category', 'champ_2': 'category', 'champ_3': 'category', 'champ_4': 'category', 'champ_5': 'category', 'champ_6': 'category', 'champ_7': 'category', 'champ_8': 'category', 'champ_9': 'category', 'champ_10': 'category', 'rank_1': 'category', 'rank_2': 'category', 'rank_3': 'category', 'rank_4': 'category', 'rank_5': 'category', 'rank_6': 'category', 'rank_7': 'category', 'rank_8': 'category', 'rank_9': 'category', 'rank_10': 'category', 'rankWins_1': 'uint32', 'rankWins_2': 'uint32', 'rankWins_3': 'uint32', 'rankWins_4': 'uint32', 'rankWins_5': 'uint32', 'rankWins_6': 'uint32', 'rankWins_7': 'uint32', 'rankWins_8': 'uint32', 'rankWins_9': 'uint32', 'rankWins_10': 'uint32', 'rankLosses_1': 'uint32', 'rankLosses_2': 'uint32', 'rankLosses_3': 'uint32', 'rankLosses_4': 'uint32', 'rankLosses_5': 'uint32', 'rankLosses_6': 'uint32', 'rankLosses_7': 'uint32', 'rankLosses_8': 'uint32', 'rankLosses_9': 'uint32', 'rankLosses_10': 'uint32', 'champPts_1': 'uint64', 'champPts_2': 'uint64', 'champPts_3': 'uint64', 'champPts_4': 'uint64', 'champPts_5': 'uint64', 'champPts_6': 'uint64', 'champPts_7': 'uint64', 'champPts_8': 'uint64', 'champPts_9': 'uint64', 'champPts_10': 'uint64', 'spell1Id_1': 'category', 'spell1Id_2': 'category', 'spell1Id_3': 'category', 'spell1Id_4': 'category', 'spell1Id_5': 'category', 'spell1Id_6': 'category', 'spell1Id_7': 'category', 'spell1Id_8': 'category', 'spell1Id_9': 'category', 'spell1Id_10': 'category', 'spell2Id_1': 'category', 'spell2Id_2': 'category', 'spell2Id_3': 'category', 'spell2Id_4': 'category', 'spell2Id_5': 'category', 'spell2Id_6': 'category', 'spell2Id_7': 'category', 'spell2Id_8': 'category', 'spell2Id_9': 'category', 'spell2Id_10': 'category', 'perk0_1': 'category', 'perk0_2': 'category', 'perk0_3': 'category', 'perk0_4': 'category', 'perk0_5': 'category', 'perk0_6': 'category', 'perk0_7': 'category', 'perk0_8': 'category', 'perk0_9': 'category', 'perk0_10': 'category', 'perk1_1': 'category', 'perk1_2': 'category', 'perk1_3': 'category', 'perk1_4': 'category', 'perk1_5': 'category', 'perk1_6': 'category', 'perk1_7': 'category', 'perk1_8': 'category', 'perk1_9': 'category', 'perk1_10': 'category', 'perk2_1': 'category', 'perk2_2': 'category', 'perk2_3': 'category', 'perk2_4': 'category', 'perk2_5': 'category', 'perk2_6': 'category', 'perk2_7': 'category', 'perk2_8': 'category', 'perk2_9': 'category', 'perk2_10': 'category', 'perk3_1': 'category', 'perk3_2': 'category', 'perk3_3': 'category', 'perk3_4': 'category', 'perk3_5': 'category', 'perk3_6': 'category', 'perk3_7': 'category', 'perk3_8': 'category', 'perk3_9': 'category', 'perk3_10': 'category', 'perk4_1': 'category', 'perk4_2': 'category', 'perk4_3': 'category', 'perk4_4': 'category', 'perk4_5': 'category', 'perk4_6': 'category', 'perk4_7': 'category', 'perk4_8': 'category', 'perk4_9': 'category', 'perk4_10': 'category', 'perk5_1': 'category', 'perk5_2': 'category', 'perk5_3': 'category', 'perk5_4': 'category', 'perk5_5': 'category', 'perk5_6': 'category', 'perk5_7': 'category', 'perk5_8': 'category', 'perk5_9': 'category', 'perk5_10': 'category', 'gameVersion': 'category', 'blueTeamWin': 'bool', 'primaryClass_1': 'category', 'secondaryClass_1': 'category', 'primaryClass_2': 'category', 'secondaryClass_2': 'category', 'primaryClass_3': 'category', 'secondaryClass_3': 'category',  'primaryClass_4': 'category', 'secondaryClass_4': 'category', 'primaryClass_5': 'category', 'secondaryClass_5': 'category', 'primaryClass_6': 'category', 'secondaryClass_6': 'category', 'primaryClass_7': 'category', 'secondaryClass_7': 'category','primaryClass_8': 'category', 'secondaryClass_8': 'category', 'primaryClass_9': 'category', 'secondaryClass_9': 'category', 'primaryClass_10': 'category', 'secondaryClass_10': 'category'})

In [ ]:
df1.info(verbose=True, null_counts=True)

In [10]:
rankMap = dict(enumerate(df1['rank_1'].cat.categories))

In [11]:
df1['avgRank1'] = ((df1['rank_1'].cat.codes.astype(int) + df1['rank_2'].cat.codes.astype(int) + df1['rank_3'].cat.codes.astype(int) + df1['rank_4'].cat.codes.astype(int) + df1['rank_5'].cat.codes.astype(int)) // (df1['rank_1'].cat.codes.astype(bool).astype(int) + df1['rank_2'].cat.codes.astype(bool).astype(int) + df1['rank_3'].cat.codes.astype(bool).astype(int) + df1['rank_4'].cat.codes.astype(bool).astype(int) + df1['rank_5'].cat.codes.astype(bool).astype(int))).replace(rankMap)
df1['avgRank2'] = ((df1['rank_6'].cat.codes.astype(int) + df1['rank_7'].cat.codes.astype(int) + df1['rank_8'].cat.codes.astype(int) + df1['rank_9'].cat.codes.astype(int) + df1['rank_10'].cat.codes.astype(int)) // (df1['rank_6'].cat.codes.astype(bool).astype(int) + df1['rank_7'].cat.codes.astype(bool).astype(int) + df1['rank_8'].cat.codes.astype(bool).astype(int) + df1['rank_9'].cat.codes.astype(bool).astype(int) + df1['rank_10'].cat.codes.astype(bool).astype(int))).replace(rankMap)

In [12]:
df1['avgRank1'] = pd.Categorical(df1['avgRank1'], ['UNRANKED', 'IRON IV', 'IRON III', 'IRON II', 'IRON I', 'BRONZE IV', 'BRONZE III', 'BRONZE II', 'BRONZE I', 'SILVER IV', 'SILVER III', 'SILVER II', 'SILVER I', 'GOLD IV', 'GOLD III', 'GOLD II', 'GOLD I', 'PLATINUM IV', 'PLATINUM III', 'PLATINUM II', 'PLATINUM I', 'DIAMOND IV', 'DIAMOND III', 'DIAMOND II', 'DIAMOND I', 'MASTER I', 'GRANDMASTER I', 'CHALLENGER I'], ordered=True)
df1['avgRank2'] = pd.Categorical(df1['avgRank2'], ['UNRANKED', 'IRON IV', 'IRON III', 'IRON II', 'IRON I', 'BRONZE IV', 'BRONZE III', 'BRONZE II', 'BRONZE I', 'SILVER IV', 'SILVER III', 'SILVER II', 'SILVER I', 'GOLD IV', 'GOLD III', 'GOLD II', 'GOLD I', 'PLATINUM IV', 'PLATINUM III', 'PLATINUM II', 'PLATINUM I', 'DIAMOND IV', 'DIAMOND III', 'DIAMOND II', 'DIAMOND I', 'MASTER I', 'GRANDMASTER I', 'CHALLENGER I'], ordered=True)

In [13]:
df1.fillna(value={'avgRank1': 'UNRANKED', 'avgRank2': 'UNRANKED'}, inplace=True)

In [14]:
for r in ['rank_1', 'rank_2', 'rank_3', 'rank_4', 'rank_5']:
    df1.loc[df1[r] == 'UNRANKED', r] = df1.loc[df1[r] == 'UNRANKED', 'avgRank1']
for r in ['rank_6', 'rank_7', 'rank_8', 'rank_9', 'rank_10']:
    df1.loc[df1[r] == 'UNRANKED', r] = df1.loc[df1[r] == 'UNRANKED', 'avgRank2']

In [ ]:
columns = ['champ_1', 'champ_2', 'champ_3', 'champ_4', 'champ_5', 'champ_6', 'champ_7', 'champ_8', 'champ_9', 'champ_10', 'rank_1', 'rank_2', 'rank_3', 'rank_4', 'rank_5', 'rank_6', 'rank_7', 'rank_8', 'rank_9', 'rank_10', 'rankWins_1', 'rankWins_2', 'rankWins_3', 'rankWins_4', 'rankWins_5', 'rankWins_6', 'rankWins_7', 'rankWins_8', 'rankWins_9', 'rankWins_10', 'rankLosses_1', 'rankLosses_2', 'rankLosses_3', 'rankLosses_4', 'rankLosses_5', 'rankLosses_6', 'rankLosses_7', 'rankLosses_8', 'rankLosses_9', 'rankLosses_10', 'champPts_1', 'champPts_2', 'champPts_3', 'champPts_4', 'champPts_5', 'champPts_6', 'champPts_7', 'champPts_8', 'champPts_9', 'champPts_10', 'spell1Id_1', 'spell1Id_2', 'spell1Id_3', 'spell1Id_4', 'spell1Id_5', 'spell1Id_6', 'spell1Id_7', 'spell1Id_8', 'spell1Id_9', 'spell1Id_10', 'spell2Id_1', 'spell2Id_2', 'spell2Id_3', 'spell2Id_4', 'spell2Id_5', 'spell2Id_6', 'spell2Id_7', 'spell2Id_8', 'spell2Id_9', 'spell2Id_10', 'perk0_1', 'perk0_2', 'perk0_3', 'perk0_4', 'perk0_5', 'perk0_6', 'perk0_7', 'perk0_8', 'perk0_9', 'perk0_10', 'perk1_1', 'perk1_2', 'perk1_3', 'perk1_4', 'perk1_5', 'perk1_6', 'perk1_7', 'perk1_8', 'perk1_9', 'perk1_10', 'perk2_1', 'perk2_2', 'perk2_3', 'perk2_4', 'perk2_5', 'perk2_6', 'perk2_7', 'perk2_8', 'perk2_9', 'perk2_10', 'perk3_1', 'perk3_2', 'perk3_3', 'perk3_4', 'perk3_5', 'perk3_6', 'perk3_7', 'perk3_8', 'perk3_9', 'perk3_10', 'perk4_1', 'perk4_2', 'perk4_3', 'perk4_4', 'perk4_5', 'perk4_6', 'perk4_7', 'perk4_8', 'perk4_9', 'perk4_10', 'perk5_1', 'perk5_2', 'perk5_3', 'perk5_4', 'perk5_5', 'perk5_6', 'perk5_7', 'perk5_8', 'perk5_9', 'perk5_10', 'gameVersion', 'blueTeamWin']
for c in columns:
    print(f"---- {c} ---")
    print(df1[c].value_counts())

In [ ]:
# Target/Frequency Encoding
# Tree Based: CatBoost, Random Forest
# Natural groupings/bins

In [15]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
# from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score

In [ ]:
# categorical_col = ['gameVersion']
categorical_col = ['rank_1', 'rank_2', 'rank_3', 'rank_4', 'rank_5', 'rank_6', 'rank_7', 'rank_8', 'rank_9', 'rank_10', 'spell1Id_1', 'spell1Id_2', 'spell1Id_3', 'spell1Id_4', 'spell1Id_5', 'spell1Id_6', 'spell1Id_7', 'spell1Id_8', 'spell1Id_9', 'spell1Id_10', 'spell2Id_1', 'spell2Id_2', 'spell2Id_3', 'spell2Id_4', 'spell2Id_5', 'spell2Id_6', 'spell2Id_7', 'spell2Id_8', 'spell2Id_9', 'spell2Id_10', 'perk0_1', 'perk0_2', 'perk0_3', 'perk0_4', 'perk0_5', 'perk0_6', 'perk0_7', 'perk0_8', 'perk0_9', 'perk0_10', 'perk1_1', 'perk1_2', 'perk1_3', 'perk1_4', 'perk1_5', 'perk1_6', 'perk1_7', 'perk1_8', 'perk1_9', 'perk1_10', 'perk2_1', 'perk2_2', 'perk2_3', 'perk2_4', 'perk2_5', 'perk2_6', 'perk2_7', 'perk2_8', 'perk2_9', 'perk2_10', 'perk3_1', 'perk3_2', 'perk3_3', 'perk3_4', 'perk3_5', 'perk3_6', 'perk3_7', 'perk3_8', 'perk3_9', 'perk3_10', 'perk4_1', 'perk4_2', 'perk4_3', 'perk4_4', 'perk4_5', 'perk4_6', 'perk4_7', 'perk4_8', 'perk4_9', 'perk4_10', 'perk5_1', 'perk5_2', 'perk5_3', 'perk5_4', 'perk5_5', 'perk5_6', 'perk5_7', 'perk5_8', 'perk5_9', 'perk5_10', 'gameVersion', 'avgRank1', 'avgRank2', 'primaryClass_1', 'secondaryClass_1', 'primaryClass_2', 'secondaryClass_2', 'primaryClass_3', 'secondaryClass_3',  'primaryClass_4', 'secondaryClass_4', 'primaryClass_5', 'secondaryClass_5', 'primaryClass_6', 'secondaryClass_6', 'primaryClass_7', 'secondaryClass_7','primaryClass_8', 'secondaryClass_8', 'primaryClass_9', 'secondaryClass_9', 'primaryClass_10', 'secondaryClass_10']

In [ ]:
le = LabelEncoder()
for column in categorical_col:
    df1[column] = le.fit_transform(df1[column])

In [16]:
for c in ['rank_1', 'rank_2', 'rank_3', 'rank_4', 'rank_5', 'rank_6', 'rank_7', 'rank_8', 'rank_9', 'rank_10', 'spell1Id_1', 'spell1Id_2', 'spell1Id_3', 'spell1Id_4', 'spell1Id_5', 'spell1Id_6', 'spell1Id_7', 'spell1Id_8', 'spell1Id_9', 'spell1Id_10', 'spell2Id_1', 'spell2Id_2', 'spell2Id_3', 'spell2Id_4', 'spell2Id_5', 'spell2Id_6', 'spell2Id_7', 'spell2Id_8', 'spell2Id_9', 'spell2Id_10', 'perk0_1', 'perk0_2', 'perk0_3', 'perk0_4', 'perk0_5', 'perk0_6', 'perk0_7', 'perk0_8', 'perk0_9', 'perk0_10', 'perk1_1', 'perk1_2', 'perk1_3', 'perk1_4', 'perk1_5', 'perk1_6', 'perk1_7', 'perk1_8', 'perk1_9', 'perk1_10', 'perk2_1', 'perk2_2', 'perk2_3', 'perk2_4', 'perk2_5', 'perk2_6', 'perk2_7', 'perk2_8', 'perk2_9', 'perk2_10', 'perk3_1', 'perk3_2', 'perk3_3', 'perk3_4', 'perk3_5', 'perk3_6', 'perk3_7', 'perk3_8', 'perk3_9', 'perk3_10', 'perk4_1', 'perk4_2', 'perk4_3', 'perk4_4', 'perk4_5', 'perk4_6', 'perk4_7', 'perk4_8', 'perk4_9', 'perk4_10', 'perk5_1', 'perk5_2', 'perk5_3', 'perk5_4', 'perk5_5', 'perk5_6', 'perk5_7', 'perk5_8', 'perk5_9', 'perk5_10', 'gameVersion', 'avgRank1', 'avgRank2', 'primaryClass_1', 'secondaryClass_1', 'primaryClass_2', 'secondaryClass_2', 'primaryClass_3', 'secondaryClass_3',  'primaryClass_4', 'secondaryClass_4', 'primaryClass_5', 'secondaryClass_5', 'primaryClass_6', 'secondaryClass_6', 'primaryClass_7', 'secondaryClass_7','primaryClass_8', 'secondaryClass_8', 'primaryClass_9', 'secondaryClass_9', 'primaryClass_10', 'secondaryClass_10']:
    df1[c] = df1[c].cat.codes
df1['blueTeamWin'] = df1['blueTeamWin'].astype(int)

In [17]:
X = df1.drop(['gameId', 'blueTeamWin'], axis=1)
y = df1['blueTeamWin']

In [18]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=2)

In [19]:
# del c
# del columns
# del categorical_col
# del column
del df1
del champions
# del X
# del y

In [20]:
import gc
gc.collect()
# gc.collect()

0

In [21]:
from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout
from keras.optimizers import SGD
from keras.layers.normalization import BatchNormalization

Using TensorFlow backend.


In [22]:
model = Sequential()
model.add(Dense(128, use_bias=False, input_shape = (X_train.shape[1],)))
model.add(BatchNormalization())
model.add(Activation("relu"))
model.add(Dropout(0.2))
model.add(Dense(64))
model.add(BatchNormalization())
model.add(Activation("relu"))
model.add(Dropout(0.2))
model.add(Dense(32))
model.add(BatchNormalization())
model.add(Activation("relu"))
model.add(Dropout(0.2))
model.add(Dense(16))
model.add(BatchNormalization())
model.add(Activation("relu"))
model.add(Dropout(0.2))
model.add(Dense(1, activation="sigmoid"))
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_1 (Dense)              (None, 128)               19584     
_________________________________________________________________
batch_normalization_1 (Batch (None, 128)               512       
_________________________________________________________________
activation_1 (Activation)    (None, 128)               0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 128)               0         
_________________________________________________________________
dense_2 (Dense)              (None, 64)                8256      
_________________________________________________________________
batch_normalization_2 (Batch (None, 64)                256       
_________________________________________________________________
activation_2 (Activation)    (None, 64)               

In [ ]:
model = Sequential()
model.add(Dense(128, activation="relu", input_shape = (X_train.shape[1],)))
model.add(BatchNormalization())
model.add(Dropout(0.2))
model.add(Dense(64, activation="relu"))
model.add(Dropout(0.2))

model.add(Dense(32, activation="relu"))
model.add(Dropout(0.2))

model.add(Dense(16, activation="relu"))
model.add(Dropout(0.2))

model.add(Dense(1, activation="sigmoid"))

model.summary()

In [23]:
# opt = SGD(lr=0.0001)
model.compile(optimizer='adam', loss = "binary_crossentropy", metrics = ['accuracy'])

In [24]:
model.fit(X_train, y_train, batch_size=32, epochs=50)

Epoch 1/50
1709575/1709575 [==============================] - 305s 178us/step - loss: 0.6832 - accuracy: 0.5607
Epoch 2/50
1709575/1709575 [==============================] - 301s 176us/step - loss: 0.6808 - accuracy: 0.5658
Epoch 3/50
1709575/1709575 [==============================] - 299s 175us/step - loss: 0.6806 - accuracy: 0.5666
Epoch 4/50
1709575/1709575 [==============================] - 301s 176us/step - loss: 0.6802 - accuracy: 0.5672
Epoch 5/50
  80256/1709575 [>.............................] - ETA: 4:51 - loss: 0.6807 - accuracy: 0.5645

KeyboardInterrupt: 

In [ ]:
validation_loss, validation_accuracy = model.evaluate(X_test, y_test, batch_size=32)
print("Loss: "+ str(validation_loss))
print("Accuracy: "+ str(validation_accuracy))

In [ ]:
y_prob = model.predict_proba(X_test)

In [ ]:
y_prob

In [ ]:
y_pred = model.predict(X_test)

In [ ]:
y_pred

In [ ]:
y_pred1 = model.predict(X_train)
print('Model accuracy score with 100 decision-trees : {0:0.4f}'. format(accuracy_score(y_train, y_pred1)))
y_pred2 = model.predict(X_test)
print('Model accuracy score with 100 decision-trees : {0:0.4f}'. format(accuracy_score(y_test, y_pred2)))